<a href="https://colab.research.google.com/github/lucianosilva-github/compiladores/blob/main/COMPILADORES_AULA_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COMPILADORES - AULA 09**

**Prof. Luciano Silva**

**OBJETIVOS DA AULA:**

*   Conhecer o processo de geração de código intermediário
*   Conhecer o código intermediário em P-Code
*   Aplicar o design pattern Visitor para gerar código intermediário em P-Code


In [1]:
!pip install rply

**GERAÇÃO DE CÓDIGO INTERMEDIÁRIO**

Quando geramos código para um determinado processador, normalmente temos que lidar com um número pequeno e limitado de registradores. Isto pode dificultar bastante a geração direta de código para o processador porque teríamos que controlar qual dado está em qual registrador para que não haja superposição de dados.

Assim, fica mais simples gerar código de forma intermediária para uma máquina (abstrata) que tenha um número infinito de registradores. Após ter o código intermediário gerado, utilizam-se algoritmos de alocação de registradores para transformar o código intermediário em código final.

Existem várias alternativas para se gerar código intermediário: **código de triplas**, **código de quádruplas**, **P-Code**, dentre outros. Na nossa disciplina, trabalharemos com P-Code.

**MÁQUINA ABSTRATA P-CODE**

O UCSD p-System era o sistema operacional escrito em Pascal da Universidade da Califórnia (University of California Software Distribution - Pascal System). Consistia em um sistema operacional que executava programas em pseudo-código, chamados de p-code, em uma máquina virtual previamente escritos em Pascal.

Era um sistema operacional que era muito popular no início dos computadores pessoais, por volta do final de 1970 e início de 1980.

Similarmente ao Java nos dia de hoje, foi baseado em uma máquina virtual com um conjunto padrão de instruções de baixo nível. As instruções em pseudo-linguagem de máquina "p-code"(como os bytecodes) eram emulados em hardware diferentes, incluindo os micro-computadores 6502, o 8080, o Z-80, e o PDP-11. Desta forma, o compillador Pascal a partir do p-code poderia gerar um programa que funcionaria em qualquer sistema P(P-System) operando em um Apple Inc. II, um Xerox 820, ou um DEC PDP-11.

A linguagem mais popular para o Sistema P era o UCSD Pascal. Na verdade, o sistema operacional P era, ele mesmo, escrito em UCSD Pascal, tornando assim o sistema operacional inteiro relativamente portátil entre plataformas diferentes. O p-System foi também um dos três sistemas operacionais originais do IBM PC, mas perdeu espaço para o MS DOS devido a, entre outros fatores, problemas comerciais e de licenciamento.

Abaixo, temos um resumo de funcionamento das máquinas P-CODE, que utiliza uma pilha s para armazenar dados:

https://homepages.cwi.nl/~steven/pascal/book/10pcode.html

Uma implementação em Pascal de uma máquina P-Code pode ser encontrada no link abaixo:

https://en.wikipedia.org/wiki/P-code_machine

Para a aula de hoje, vamos usar as instruções abaixo:

* **lit 0 a** : empilha a constante a na pilha

* **opr 0 a**  : executa a operação a
  
            1: s[t] := -s[t]
            2: t := t - 1 s[t] := s[t] + s[t + 1] 
            3: t := t - 1 s[t] := s[t] - s[t + 1] 
            4: t := t - 1 s[t] := s[t] * s[t + 1] 
            5: t := t - 1 s[t] := s[t] div s[t + 1] 

* **lod L a**  : empilha a variável do nível L a na pilha

* **sto L a**  : desempilha o topo da pilha, armazenando-a na variável a no nível L

* **cal L a**  : invoca o procedimento a no nível L

* **int 0 a**  : incrementa o topo da pilha (t) de a (t=t+a)

* **jmp 0 a**  : pula para o endereço a

* **jpc 0 a**  : pula condicionalmente para o endereço a

Abaixo, temos exemplo de geração de código:

* **3+5**
  
  lit 0 3

  lit 0 5

  opr 0 2
  
* **3*5+6/2**
  
  lit 0 3

  lit 0 5

  opr 0 4

  lit 0 6

  lit 0 2

  opr 0 5

  opr 0 2

* **y=x+2**
  
  lod 0 x

  lit 0 2

  opr 0 2

  sto 0 y


**EXERCÍCIO**

Implementar um visitor para gerar código intermediário em P-Code para a regra **\< expression \>** :


\<prog\> ::= \<var-decls\> \<atrib\>

\<var-decls\> ::= \<var-decl\> 

       | \<var-decl\> \<var-decls\>

\<var-decl\> ::= \<type\> ID ;

\<type\> ::= int | string 

\<atrib\> ::= ID = \<expression\>

\<expression\> ::= ID

      | NUMBER

      | \<expression\> "+" \<expression\>
 
      | \<expression\> "-" \<expression\>
 
      | \<expression\> "*" \<expression\>
 
      | \<expression\> "/" \<expression\>
 
      | "(" <expression> ")"

* Implementação do analisador léxico:





In [2]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('INT', r'int')
lg.add('STRING', r'string')
lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')

lg.ignore('\s+')

lexer = lg.build()

* Classes dos nós da árvore sintática, já com o método accept para receber os visitors:

In [3]:
from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,atrib):
        self.decls = decls
        self.atrib = atrib

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp
        

    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass
    

class Sub(BinaryOp):
  pass
   

class Mul(BinaryOp):
  pass
    

class Div(BinaryOp):
  pass
    

* Implementação do analisador sintático:

In [4]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'STRING', 'ID','SEMICOL',
     'EQUALS'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])    
    ]
)

@pg.production('prog : vardecls atrib')
def prog(p):
    return Prog(p[0],p[1])

@pg.production('vardecls : vardecl')
def expression_vardeclsOne(p):
    return VarDecls(p[0],None) 

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : STRING ID SEMICOL')
def expression_vardeclstring(p):
    return VarDecl(p[1].getstr(), p[0].getstr())

@pg.production('vardecl : INT ID SEMICOL')
def expression_vardeclint(p):
    return VarDecl(p[1].getstr(), p[0].getstr())

@pg.production('atrib : ID EQUALS expression')
def atrib(p):
    return Atrib(p[0].getstr(),p[2])


@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [7]:
arvore=parser.parse(lexer.lex('int x;int y;int z;z=x+y'))

* Visitor para montar a tabela de símbolos:

In [8]:
ST={}

class Visitor(object):
  pass

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)
        
    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id]=d.tp
        

In [9]:
arvore.accept(SymbolTable())
for k, v in ST.items():
  print(k, v)

x int
y int
z int


* Visitor para decoração da árvore: 

In [11]:
class Decorator(Visitor):

    def visit_prog(self, i):
        i.atrib.accept(self)

    def visit_atrib(self, i):
        if i.id in ST:
          i.id_decor_type=ST[i.id]
        else:
          raise AssertionError('id not declared')
        i.expr.accept(self)
        i.expr_decor_type=i.expr.decor_type

    def visit_id(self, i):
        if i.value in ST:
          i.decor_type=ST[i.value]
        else:
          raise AssertionError('id not declared')


    def visit_number(self, i):
        i.decor_type="int"
        

    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"
          

    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="int" and a.right.decor_type=="int":
          a.decor_type="int"

In [12]:
arvore.accept(Decorator())

* Visitor para verificação de tipos:

In [13]:
class TypeVerifier(Visitor):

    def visit_prog(self, i):
        i.atrib.accept(self)

    def visit_atrib(self, i):
        if i.id_decor_type!=i.expr_decor_type:
          raise AssertionError('type error')

    def visit_add(self, a):
        if a.left.decor_type != a.right.decor_type:
          raise AssertionError('type error')
          

    def visit_sub(self, a):
        if a.left.decor_type != a.right.decor_type:
          raise AssertionError('type error')

    def visit_mul(self, a):
        if a.left.decor_type != a.right.decor_type:
          raise AssertionError('type error')

    def visit_div(self, a):
        if a.left.decor_type != a.right.decor_type:
          raise AssertionError('type error')

In [14]:
arvore.accept(TypeVerifier())

In [ ]:
class IntermediateCode(Visitor):
  #Implemente seu visitor aqui. Para cada instrução intermediária gerada, ela deve ser mostrada na tela.

In [ ]:
arvore.accept(IntermediateCode())

**ATIVIDADE EAD**

Aumentar o visitor de geração de código intermediário para gerar cógio para atribuições (regra \<attrib \>).



In [ ]:
#Implemente sua solução aqui